
$$
\begin{aligned}
& K_{\text{fratio}} = \frac{1}{1 + K_{f} \cdot q_{\text{fpure}}} \\
& K_{\text{rratio}} = \frac{1}{1 - K_{r} \cdot q_{\text{rpure}}} \\
& q_{\text{fpure}} = q_{\text{ftp30}} + q_{\text{froll}^{\prime}} \\
& q_{\text{froll}^{\prime}} = \frac{q_{\text{froll}}}{100} \times \operatorname{\theta r} \times \frac{1}{0.5 \times 9.8 \times \frac{m_{f}}{2}} \times 10^{6} \\
& q_{\text{rpure}} = q_{\text{rtp30}} + q_{\text{rroll}^{\prime}} \\
& q_{\text{rroll}^{\prime}} = \frac{q_{\text{rroll}}}{100} \times \operatorname{\theta r} \times \frac{1}{0.5 \times 9.8 \times \frac{m_{r}}{2}} \times 10^{6}
\end{aligned}
$$

In [1]:
import json
from ipywidgets import interactive, Button, HBox, VBox, IntSlider, FloatSlider, Output
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ---unique--------------------------------
output = Output()
# Global variable
FRATIO_LEVELS = [0.80, 0.85, 0.90]

def interactive_plot_func(mf,theta_r):
    theta_r = theta_r / 180*np.pi
    
    q_froll = np.linspace(-1, 15, 100) / 100  # [%] --> [-]
    q_ftp30 = np.linspace(-100, 200, 100) * 10**-6 * np.pi/180  # [μdeg/N] --> [rad/N]

    q_froll, q_ftp30 = np.meshgrid(q_froll, q_ftp30)

    q_froll_prime = q_froll * theta_r * 1/(0.5 * 9.8 * mf/2)
    q_fpure = q_ftp30 + q_froll_prime

    K_f = 1000*180/np.pi
    K_fratio = 1 / (1 + K_f * q_fpure)

    fig, ax = plt.subplots()
    
    
    # Functions to customize label formats
    def format_Kfratio_label(x):
        return f"Kfratio={x*100:.1f}"

    dataframes = []
    for fratio in FRATIO_LEVELS:
        contour = ax.contour(q_ftp30*10**6/np.pi*180, q_froll*100, K_fratio, levels=[fratio])
        ax.clabel(contour, inline=1, fontsize=10, fmt=format_Kfratio_label)
        for path in contour.collections[0].get_paths():
            for point in path.vertices:
                dataframes.append({'q_ftp30': point[0], 'q_froll': point[1], 'K_fratio': fratio})
    ax.set_xlabel('roll steer $q_{ftp30}$ [μdeg/N]')
    ax.set_ylabel('compliance steer $q_{froll}$ [%]')
    ax.set_title('Front equivalent CP')
    ax.grid()
            
    plt.show()

    with output:
        output.clear_output()  # Clear the previous value
        print(f"K_fratio min: {K_fratio.min() * 100:.1f}, max: {K_fratio.max() * 100:.1f}")
    display(output)
    
    df_all = pd.DataFrame(dataframes)
    return df_all

def save_data(button):
    df_all = interactive_plot_func(**{child.description.split(' ')[0]: child.value for child in interactive_plot.children[:-1]})
    
    # Create a new dataframe to store q_ftp30 and q_froll values for each K_fratio
    df_to_save = pd.DataFrame()

    for fratio in FRATIO_LEVELS:
        df_fratio = df_all[df_all["K_fratio"] == fratio]
        df_ftp30 = df_fratio[['q_ftp30']].rename(columns={"q_ftp30": f"q_ftp30_K_fratio={fratio}"}).reset_index(drop=True)
        df_froll = df_fratio[['q_froll']].rename(columns={"q_froll": f"q_froll_K_fratio={fratio}"}).reset_index(drop=True)
        df_to_save = pd.concat([df_to_save, df_ftp30, df_froll], axis=1)

    df_to_save.to_csv('data.csv', index=False)



# ---partially unique--------------------------------

# define a dictionary that links the description to the name and unit
slider_parameters = {
    "mf": {"name": "Front mass", "unit": "kg"},
    "theta_r": {"name": "roll ratio", "unit": "deg"},
}

# Creating the sliders
spec_sliders = {
    "mf": IntSlider(min=500, max=2000, step=10, value=1000, description=f'mf [{slider_parameters["mf"]["unit"]}]', continuous_update=False),
    "theta_r": FloatSlider(min=1.0, max=3.0, step=0.1, value=2.5, description=f'theta_r [{slider_parameters["theta_r"]["unit"]}]', continuous_update=False),
}
condition_sliders = {
}

# ---common--------------------------------

interactive_plot = interactive(interactive_plot_func,
                               **spec_sliders, 
                               **condition_sliders)

# Function to define the operation of the Save button
def save_values(b):
    spec_slider_values = {
        key: {
            'name': slider_parameters[key]['name'],
            'value': slider.value,
            'unit': slider_parameters[key]['unit']
        }
        for key, slider in spec_sliders.items()
    }
    try:
        with open('spec_defaults.json', 'r+') as f:
            data = json.load(f)
            data.update(spec_slider_values)
            f.seek(0)
            f.write(json.dumps(data))
            f.truncate()
    except FileNotFoundError:
        with open('spec_defaults.json', 'w') as f:
            json.dump(spec_slider_values, f)
            
# Function to define the operation of the Load button
def load_values(b):
    try:
        with open('spec_defaults.json', 'r') as f:
            defaults = json.load(f)
            for key, slider in spec_sliders.items():
                if key in defaults:
                    slider.value = defaults[key]["value"]
    except FileNotFoundError:
        print("No default values found. Please save first.")

# create buttons
save_param_button = Button(description="Save parameters")
save_param_button.on_click(save_values)

load_param_button = Button(description="Load parameters")
load_param_button.on_click(load_values)

save_data_button = Button(description="Save data")
save_data_button.on_click(save_data)

# Displaying the UI elements
display(HBox([interactive_plot, VBox([save_param_button, load_param_button, save_data_button])]))